In [1]:
!!pip install gradio tensorflow numpy pandas

['Requirement already satisfied: gradio in d:\\anaconda\\envs\\drqn-env\\lib\\site-packages (4.44.1)',
 'Requirement already satisfied: tensorflow in d:\\anaconda\\envs\\drqn-env\\lib\\site-packages (2.10.1)',
 'Requirement already satisfied: numpy in d:\\anaconda\\envs\\drqn-env\\lib\\site-packages (1.23.5)',
 'Requirement already satisfied: pandas in d:\\anaconda\\envs\\drqn-env\\lib\\site-packages (2.2.3)',
 'Requirement already satisfied: aiofiles<24.0,>=22.0 in d:\\anaconda\\envs\\drqn-env\\lib\\site-packages (from gradio) (23.2.1)',
 'Requirement already satisfied: anyio<5.0,>=3.0 in d:\\anaconda\\envs\\drqn-env\\lib\\site-packages (from gradio) (4.6.2)',
 'Requirement already satisfied: fastapi<1.0 in d:\\anaconda\\envs\\drqn-env\\lib\\site-packages (from gradio) (0.115.12)',
 'Requirement already satisfied: ffmpy in d:\\anaconda\\envs\\drqn-env\\lib\\site-packages (from gradio) (0.5.0)',
 'Requirement already satisfied: gradio-client==1.3.0 in d:\\anaconda\\envs\\drqn-env\\lib\

In [2]:
import gradio as gr
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import random

# ✅ GPU configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU is enabled and memory growth set.")
    except RuntimeError as e:
        print(f"GPU setup error: {e}")
else:
    print("❌ GPU not found. Using CPU.")


D:\Anaconda\envs\drqn-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ GPU is enabled and memory growth set.


In [3]:


categories = ['Politics', 'Tech', 'Sports', 'Entertainment', 'Health', 'Science', 'Finance', 'Lifestyle', 'Travel', 'Education']

news_df = pd.DataFrame({
    "id": list(range(50)),
    "title": [
        f"{random.choice(['Breaking', 'Latest', 'Update'])}: {random.choice(['AI', 'Elections', 'Health', 'Stocks', 'Space', 'Movies', 'Football', 'Travel', 'Nutrition'])} News {i+1}"
        for i in range(50)
    ],
    "category": [random.choice(categories) for _ in range(50)]
})

# ✅ For tracking user interaction
click_counts = [0] * len(news_df)



In [4]:
# RL parameters
state_size = len(news_df)  # current state = user interaction vector
action_size = len(news_df)  # actions = articles to recommend

# Build model
def create_model():
    model = tf.keras.Sequential([
        layers.Dense(64, input_dim=state_size, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(action_size, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Initialize policy and target networks
policy_model = create_model()
target_model = create_model()
target_model.set_weights(policy_model.get_weights())


In [5]:
# Experience Replay Buffer
class ReplayBuffer:
    def __init__(self, max_size=1000):
        self.buffer = []
        self.max_size = max_size

    def add(self, s, a, r, s2, done):
        if len(self.buffer) >= self.max_size:
            self.buffer.pop(0)
        self.buffer.append((s, a, r, s2, done))

    def sample(self, batch_size):
        return random.sample(self.buffer, min(len(self.buffer), batch_size))

buffer = ReplayBuffer()

def train(batch_size=32, gamma=0.99):
    samples = buffer.sample(batch_size)
    for state, action, reward, next_state, done in samples:
        q_update = reward
        if not done:
            q_update += gamma * np.max(target_model.predict(np.array([next_state]))[0])
        q_values = policy_model.predict(np.array([state]))[0]
        q_values[action] = q_update
        with tf.device('/GPU:0'):
            policy_model.fit(np.array([state]), np.array([q_values]), epochs=1, verbose=0)


In [6]:
user_state = np.zeros(state_size)

def recommend(state=None):
    if state is None:
        state = user_state
    q_values = policy_model.predict(np.array([state]))[0]
    sorted_indices = np.argsort(q_values)[::-1]
    top_5 = sorted_indices[:5]
    button_texts = [f"{news_df.iloc[i]['title']} ({news_df.iloc[i]['category']})" for i in top_5]
    return top_5, button_texts


In [7]:
def on_click(news_title, user_state):
    idx = news_df[news_df['title'].str.contains(news_title.split(" (")[0])].index[0]
    reward = 1
    click_counts[idx] += 1

    next_state = user_state.copy()
    next_state[idx] = 1
    buffer.add(user_state, idx, reward, next_state, False)

    train()

    _, new_titles = recommend(next_state)

    return (
        f"You clicked: {news_title}",
        *[gr.update(value=news) for news in new_titles],
        next_state
    )


In [8]:
import gradio as gr

with gr.Blocks() as demo:
    state = gr.State(value=user_state)
    output = gr.Textbox(label="Clicked Article Info")

    # 5 Fixed Buttons
    btn1 = gr.Button()
    btn2 = gr.Button()
    btn3 = gr.Button()
    btn4 = gr.Button()
    btn5 = gr.Button()
    buttons = [btn1, btn2, btn3, btn4, btn5]

    # Initial recommendation titles
    _, initial_titles = recommend()
    for btn, title in zip(buttons, initial_titles):
        btn.value = title

    # Outputs: textbox + updated buttons + updated state
    outputs = [output] + buttons + [state]

    # Hook every button with click logic
    for btn in buttons:
        btn.click(on_click, inputs=[btn, state], outputs=outputs)

demo.launch()


1/1 [==============================] - 3s 3s/step
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 49ms/step
